<a href="https://colab.research.google.com/github/muskaanfayyaz/Support-Agent-System/blob/main/Support_Agent_System_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -Uq openai-agents
!pip install -Uq openai openai-agents

In [ ]:
import importlib.util
print(importlib.util.find_spec("openai_agents"))

In [ ]:
from agents import Agent, Runner
print("✅ openai_agents imported successfully!")


In [ ]:
import nest_asyncio
nest_asyncio.apply()
from google.colab import userdata

GEMINI_API_KEY = userdata.get("GEMINI_API_KEY")
if not GEMINI_API_KEY:
    raise ValueError("❌ No GEMINI_API_KEY found in Colab userdata. Go to Colab → Tools → Variables and set it.")


In [ ]:
# =============================
# Imports
# =============================
import asyncio
import logging
from typing import Any
from pydantic import BaseModel
from openai import AsyncOpenAI
from agents import (
    Agent,
    Runner,
    RunConfig,
    ModelSettings,
    OpenAIChatCompletionsModel,
    function_tool,
    input_guardrail,
    RunContextWrapper,
    GuardrailFunctionOutput,
    enable_verbose_stdout_logging,
)
from agents.exceptions import InputGuardrailTripwireTriggered

# =============================
# Setup + Logging
# =============================
enable_verbose_stdout_logging()
logging.basicConfig(level=logging.INFO)

# -----------------------------
# External Gemini Client + Model
# -----------------------------
external_client = AsyncOpenAI(
    api_key="YOUR_GEMINI_KEY_HERE",  # Replace with env var in production
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/",
)

model = OpenAIChatCompletionsModel(
    model="gemini-2.5-pro",
    openai_client=external_client,
)

config = RunConfig(model=model)

# =============================
# CONTEXT
# =============================
class UserContext(BaseModel):
    name: str = "Guest"
    is_premium_user: bool = False
    issue_type: str | None = None

# =============================
# GUARDRAILS
# =============================
@input_guardrail
async def no_apology_guardrail(
    ctx: RunContextWrapper[None], agent: Agent, input: str
) -> GuardrailFunctionOutput:
    """Block any input where the user says 'sorry'."""
    if "sorry" in input.lower():
        return GuardrailFunctionOutput(
            output_info="Apology detected!",
            tripwire_triggered=True,
        )
    return GuardrailFunctionOutput(output_info="ok", tripwire_triggered=False)

# =============================
# TOOLS
# =============================
@function_tool
async def refund(ctx: RunContextWrapper[UserContext]) -> str:
    user = ctx.context
    if not user.is_premium_user:
        return f"⚠️ Refunds are only available to premium users. Current user: {user.name}"
    return f"💰 Refund issued for premium user {user.name}"

@function_tool
async def restart_service(ctx: RunContextWrapper[UserContext]) -> str:
    user = ctx.context
    return f"🔄 Service restarted for {user.name}"

@function_tool
async def faq_search(ctx: RunContextWrapper[UserContext]) -> str:
    return "📖 FAQ result: Please check your network connection."

# =============================
# AGENTS
# =============================
billing_agent = Agent(
    name="BillingAgent",
    handoff_description="Handles billing and refund queries.",
    instructions="""
    You are the billing support agent.
    If a customer asks about refunds, billing, or payments:
    1. Use the refund tool.
    2. If irrelevant, transfer back to triage.
    """,
    tools=[refund],
    model_settings=ModelSettings(temperature=0.2, max_tokens=400),
)

tech_agent = Agent(
    name="TechAgent",
    handoff_description="Handles technical issues.",
    instructions="""
    You are the technical support agent.
    If a customer has a service issue:
    1. Ask clarifying questions.
    2. Use the restart service tool if needed.
    3. If irrelevant, transfer back to triage.
    """,
    tools=[restart_service],
    model_settings=ModelSettings(temperature=0.2, max_tokens=400),
)

general_agent = Agent(
    name="GeneralAgent",
    handoff_description="Handles general FAQs.",
    instructions="""
    You are the general FAQ agent.
    For general queries, use the FAQ search tool.
    """,
    tools=[faq_search],
    model_settings=ModelSettings(temperature=0.2, max_tokens=400),
)

triage_agent = Agent(
    name="TriageAgent",
    handoff_description="Delegates the user to the right support agent.",
    instructions="""
    You are the triage agent. Detect user intent:
    - If refund/billing → BillingAgent
    - If technical/service issue → TechAgent
    - Else → GeneralAgent
    """,
    handoffs=[billing_agent, tech_agent, general_agent],
    input_guardrails=[no_apology_guardrail],
    model_settings=ModelSettings(temperature=0.3, max_tokens=400),
)

# =============================
# HELPERS
# =============================
def extract_text(result: Any) -> str:
    """Extracts text from agent outputs (safe fallback)."""
    if hasattr(result, "final_output") and result.final_output:
        return str(result.final_output)
    return str(result)

# =============================
# RUN LOOP
# =============================
async def main() -> None:
    ctx = UserContext(name="Muskaan", is_premium_user=True)
    print("🤖 Support Agent System (type 'exit' to quit)\n")

    while True:
        user_input = input("You: ").strip()
        if user_input.lower() == "exit":
            break

        # Tag issue type
        low = user_input.lower()
        if any(k in low for k in ("refund", "billing", "payment")):
            ctx.issue_type = "billing"
        elif any(k in low for k in ("restart", "not working", "error", "crash", "bug", "down")):
            ctx.issue_type = "technical"
        else:
            ctx.issue_type = "general"

        try:
            result = await Runner.run(triage_agent, user_input, context=ctx, run_config=config)
            print(f"Agent: {extract_text(result)}\n")
        except InputGuardrailTripwireTriggered as e:
            print(f"🚫 Guardrail Blocked Input: {e}\n")

# =============================
# Entry Point
# =============================
if __name__ == "__main__":
    asyncio.run(main())
